In [1]:
import higlass
from higlass.client import View, Track, CombinedTrack
from higlass.tilesets import cooler, beddb, chromsizes, bigwig, Tileset
import clodius
import os
import os.path as path
import pandas as pd
import numpy as np
import itertools
import negspy.coordinates as nc

genes = beddb("Data/hg38/gene-annotations-hg38.beddb")
chrom_sizes = chromsizes("hg38.chrom.sizes")
label_font_size = 18

chrom_labels = {
    "top": Track(
        track_type="horizontal-chromosome-labels",
        tileset=chrom_sizes,
        position="top",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "bottom": Track(
        track_type="horizontal-chromosome-labels",
        tileset=chrom_sizes,
        position="bottom",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="vertical-chromosome-labels",
        tileset=chrom_sizes,
        position="left",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "right": Track(
        track_type="vertical-chromosome-labels",
        tileset=chrom_sizes,
        position="right",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
}

gene_annots = {
    "top": Track(
        track_type="horizontal-gene-annotations",
        tileset=genes,
        position="top",
        height=150,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "bottom": Track(
        track_type="horizontal-gene-annotations",
        tileset=genes,
        position="bottom",
        height=150,
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="vertical-gene-annotations",
        tileset=genes,
        position="left",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "right": Track(
        track_type="vertical-gene-annotations",
        tileset=genes,
        position="right",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
}

ctcf_motifs = {
    "top": Track(
        track_type="bedlike",
        server="//higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="top",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    ),
    "left": Track(
        track_type="bedlike",
        server="//higlass.io/api/v1",
        tileset_uuid="EkPGY0iFQx6Nq6vdF8CpWA",
        position="left",
        options={
            "fontSize": label_font_size,
            "showMousePosition": True,
        },
    )
}

hg38 = nc.get_chrominfo("hg38")

In [2]:
tumour_metadata = pd.read_csv(path.join("..", "..", "Data", "External", "LowC_Samples_Data_Available.tsv"), sep="\t", header=0)
tumour_metadata = tumour_metadata.loc[tumour_metadata.Include == "Yes", :]
tumour_metadata["SampleID"] = ["PCa" + str(i) for i in tumour_metadata["Sample ID"]]

tumour_samples = tumour_metadata["SampleID"].tolist()
t2e_samples = tumour_metadata.loc[tumour_metadata["T2E Status"] == "Yes", "SampleID"].tolist()
nont2e_samples = tumour_metadata.loc[tumour_metadata["T2E Status"] == "No", "SampleID"].tolist()

benign_metadata = pd.read_csv(path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX", "config.tsv"), sep="\t", header=0)
benign_metadata = benign_metadata.loc[benign_metadata.Include == "Yes", :]
benign_samples = benign_metadata["Sample"].tolist()

cell_line_metadata = pd.read_csv(path.join("..", "..", "Data", "External", "Rhie_2019", "config.tsv"), sep="\t", header=0)
cell_line_samples = cell_line_metadata["Run_Accession"].tolist()

all_samples = tumour_samples + benign_samples + cell_line_samples
metadata = pd.read_csv(path.join("..", "2020-01-15_TAD-aggregation", "config.tsv"), sep="\t", index_col=False, header=0)
cooler_files = (
    [path.join("..", "..", "Data", "Processed", "2019-06-18_PCa-LowC-sequencing", "Contacts", s + ".mcool") for s in tumour_samples]
    + [path.join("..", "..", "Data", "Raw", "191220_A00827_0104_AHMW25DMXX", "Contacts", s + ".mcool") for s in benign_samples]
    + [path.join("..", "..", "Data", "External", "Rhie_2019", "Contacts", s + ".mcool") for s in cell_line_samples]
)
lowc_tilesets = {s: cooler(f) for s, f in zip(all_samples, cooler_files)}

resolutions = [
    1000, 2000, 3000, 4000, 5000,
    10000, 20000, 30000, 40000, 50000,
    100000, 200000, 300000, 400000, 500000,
    1000000, 2000000, 3000000, 4000000, 5000000
][::-1]

min_resolution = 10000
heatmap_size = 150
colour_range = [
    "rgba(65, 105, 225, 1.0)",
    "rgba(255, 250, 250, 1.0)",
    "rgba(240, 128, 128, 1.0)",
    "rgba(255, 99, 71, 1.0)"
]
lowc_heatmaps = {
    s: {
        "top": Track(
            track_type='horizontal-heatmap',
            position='top',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Label"].values[0],
                "colorRange": colour_range,
            },
        ),
        "bottom": Track(
            track_type='horizontal-heatmap',
            position='bottom',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Label"].values[0],
                "colorRange": colour_range,
            }
        ),
        "left": Track(
            track_type='vertical-heatmap',
            position='left',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Label"].values[0],
                "colorRange": colour_range,
            }
        ),
        "right": Track(
            track_type='vertical-heatmap',
            position='right',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            width=heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Label"].values[0],
                "colorRange": colour_range,
            }
        ),
        "centre": Track(
            track_type='heatmap',
            position='center',
            tileset=lowc_tilesets[s],
            filetype="cooler",
            height=2 * heatmap_size,
            options={
                "maxZoom": str(resolutions.index(min_resolution)),
                "colorbarPosition": "topRight",
                "showMousePosition": True,
                "name": metadata.loc[metadata["SampleID"] == s, "Label"].values[0],
                "colorRange": colour_range,
            }
        )
    } for s in all_samples
}

metadata

,SampleID,Include,Source,Type,Label,Sample_Colour,Type_Colour,Contact_File,Tissue
0,PCa13266,Yes,Primary,Malignant,CPCG0268,#7F3C8D,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
1,PCa13848,Yes,Primary,Malignant,CPCG0366,#11A579,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
2,PCa14121,Yes,Primary,Malignant,CPCG0255,#3969AC,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
3,PCa19121,Yes,Primary,Malignant,CPCG0258,#F2B701,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
4,PCa3023,Yes,Primary,Malignant,CPCG0324,#E73F74,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
5,PCa33173,Yes,Primary,Malignant,CPCG0246,#80BA5A,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
6,PCa40507,Yes,Primary,Malignant,CPCG0342,#E68310,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
7,PCa51852,Yes,Primary,Malignant,CPCG0336,#008695,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
8,PCa53687,Yes,Primary,Malignant,CPCG0339,#CF1C90,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate
9,PCa56413,Yes,Primary,Malignant,CPCG0331,#F97B72,#1F77B4,../../Data/Processed/2019-06-18_PCa-LowC-seque...,Prostate


In [5]:
d, c, v = higlass.display(views=[])
d

HiGlassDisplay(viewconf={'editable': True, 'views': [], 'trackSourceServers': ['http://higlass.io/api/v1'], 'l…